In [ ]:
import sys
import torch
import jacinle
import json
import pickle
import numpy as np

import jactorch

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from typing import Optional, Union, List, Dict
from jacinle.config.environ_v2 import configs, set_configs
from left.domain import create_domain_from_parsing
from left.models.model import LeftModel
import re

import torch
import torch.nn as nn
from typing import Union, List, Dict
from torch.nn import TransformerEncoder, TransformerEncoderLayer

import jacinle
from jacinle.utils.container import GView
from jacinle.config.environ_v2 import configs, set_configs, def_configs
from concepts.benchmark.clevr.clevr_constants import g_attribute_concepts, g_relational_concepts
from left.models.model import LeftModel
import json
import os

from experiments.desc_neuro_codex_clevr_learned_belongings import *

In [ ]:
import jacinle.io as io
from os import path as osp

# data_dir = "data/CLEVR_CoGenT_v1.0/valB/"
data_dir = "data/clevr/val/"

data_parses = [
    f"{data_dir}/questions-ncprogram-gt.pkl"
    ]
all_parses = dict()
for filename in data_parses:
    if filename.endswith('.p'):
        content = io.load_pkl(filename)
    else:
        content = io.load(filename)
    all_parses.update(content)

data_questions_json = osp.join(data_dir, 'questions.json')
data_scenes_json = osp.join(data_dir, 'scenes.json')
data_image_root = osp.join(data_dir, 'images')
data_vocab_json = osp.join(data_dir, 'vocab.json')
data_output_vocab_json = osp.join(data_dir, 'output-vocab.json')
from left.domain import create_domain_from_parsing

domain = create_domain_from_parsing(all_parses)



from concepts.benchmark.clevr.dataset import make_dataset
validation_dataset = make_dataset(
    data_scenes_json,
    data_questions_json,
    data_image_root,
    vocab_json=data_vocab_json,
    output_vocab_json=data_output_vocab_json,
)



In [ ]:
batch_size = 1
num_workers = 96
validation_dataloader = validation_dataset.make_dataloader(batch_size, shuffle=False, drop_last=False, nr_workers=num_workers)


In [ ]:

from jacinle.config.g import g
g.concept_mapping = None
domain.print_summary()

In [ ]:
len(dataset)

In [ ]:
from jactorch.io import state_dict, load_state_dict
from left.data.referit3d.vocabulary import Vocabulary
from jactorch.train import TrainerEnv
from jactorch.optim import AdamW
from jactorch.cuda.copy import async_copy_to

vocab = Vocabulary()
model = make_model(None, domain, all_parses, validation_dataset.output_vocab if hasattr(validation_dataset, 'output_vocab') else validation_dataset.unwrapped.output_vocab, custom_transfer=None)
model.cuda()
trainable_parameters = filter(lambda x: x.requires_grad, model.parameters())

optimizer = AdamW(trainable_parameters, 0.001 , weight_decay=1e-2)

trainer = TrainerEnv(model, optimizer)

filename = "[PATH_TO_MODEL]"

trainer.load_checkpoint(filename)
model.eval()

In [ ]:

group_meters = jacinle.GroupMeters()
group_meters.reset()
from jacinle.utils.tqdm import tqdm_pbar
from jactorch.utils.meta import as_float
wrong_indices = []
correct_indices = []
parse_fails = []
with tqdm_pbar(total=len(validation_dataloader)) as pbar:
    for (index, feed_dict) in enumerate(validation_dataloader):
        feed_dict = async_copy_to(feed_dict, 0)
        output_dict, extra_info = trainer.evaluate(feed_dict)

        if output_dict["results"][0][2] is None:
            parse_fails.append(index)
            correct = False
        if output_dict["monitors"]["acc/qa"] == 1:
            correct =  True
        else:
            correct = False

        if not correct:
            wrong_indices.append(index)
        else:
            correct_indices.append(index)
            

        acc = len(correct_indices) / (len(correct_indices) + len(wrong_indices))

        pbar.set_description(group_meters.format_simple(
            f'Validation Acc: {acc}',
            {k: v for k, v in group_meters.val.items() if k.startswith('validation') and k.count('/') <= 2},
            compressed=True
        ), refresh=False)
        pbar.update()
        if index == 200:
            break
# data["program"][0] = "exists(Object, lambda x:same_row(x,iota(Object, lambda y: yellow(y))))"
# # print(data["program"])
# loss, monitors, outputs = model(data)
# print(i,outputs)

In [ ]:
parse_fails = []

In [ ]:
len(parse_fails), len(wrong_indices), len(correct_indices)

In [ ]:
wrong_indices


In [ ]:
iterator = iter(validation_dataloader)
data_all = [next(iterator) for _ in range(200)]

In [ ]:
for d in data_all:
    if d["question_raw"][0] not in all_parses:
        print("not found")

In [ ]:
%matplotlib inline
import sys

## Open a real image from address draw bouding boxes for objects
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import json
# plt.close()
from copy import deepcopy
from reascan import get_image
feed_dict = data_all[wrong_indices[11]]
feed_dict = async_copy_to(feed_dict, 0)
print(feed_dict)
q = feed_dict["question_raw"][0]
print("program:", all_parses[q])


output_dict, extra_info = trainer.evaluate(feed_dict)
image = Image.open(osp.join(data_image_root, feed_dict["image_filename"][0])).convert('RGB')
print(feed_dict["question_raw"])
for i in range(len(output_dict["execution_traces"][0])):
    
    if len(str(output_dict["execution_traces"][0][i][0])) > 6:
        tensor_values = np.array(output_dict["execution_traces"][0][i][1].tensor.detach().cpu().numpy())
        tensor_values = (tensor_values - tensor_values.min()) / (tensor_values.max() - tensor_values.min())
        fig, ax = plt.subplots()
        ax.imshow(image)
        for obj_index,obj in enumerate(feed_dict["objects_raw"][0]):
            # Create a Rectangle patch
            rect = patches.Rectangle((obj[0], obj[1]), obj[2] - obj[0], obj[3] - obj[1], linewidth=1, edgecolor='r', facecolor="none")
            ax.add_patch(rect)
            try:
                text = f"{obj_index} {round(float(tensor_values[obj_index]),4)} "
            except:
                text = f"{obj_index} {tensor_values} "
            ax.text(obj[0], obj[1], text, color='white', fontsize=10, bbox=dict(facecolor='red', alpha=0.5))
        plt.title(str(output_dict["execution_traces"][0][i][0]))
        plt.show()
        print(tensor_values)

plt.show()
print(tensor_values.argmax().item())
print(validation_dataset.unwrapped.output_vocab.idx2word[tensor_values.argmax().item()])
